In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    # 如果没有配置环境变量，请用API Key将下行替换为：api_key="sk-xxx"
    # 新加坡和北京地域的API Key不同。获取API Key：https://help.aliyun.com/zh/model-studio/get-api-key
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    # 以下是北京地域base_url，如果使用新加坡地域的模型，需要将base_url替换为：https://dashscope-intl.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
name='天虞'
category='人名'
description='有人反臂'

completion = client.chat.completions.create(
    model="qwen-max",
    messages=[
        {
            "role": "system",
            "content": "你是《山海经》研究专家和AI图像生成提示词专家。请基于形象,分类和描述生成适合AI绘画的prompt，如果有描述，详细扩写描述部分内容，不要有其他内容，如果没有则自由发挥，利用你的知识库搜索相关的神话学、考古学资料，参考古代插画风格和传统中国美学的彩色水墨风格,只返回prompt，以prompt开头。"
        },
        {
            "role": "user",
            "content": f"名字：{name}，分类：{category}，描述：{description}", 
        },
    ],
    extra_body={"enable_search": True}
)

json_string = completion.choices[0].message.content
print(json_string)

prompt: 中国传统风格的彩色水墨画，描绘了一个名叫天虞的人，他有着独特的特征——反臂。他的手臂从肩膀处向后弯曲，手肘朝前，手掌则自然地放在背后，仿佛天生如此。天虞身穿古代服饰，长袍宽袖，衣袂飘飘，展现出一种超凡脱俗的气质。背景是山川河流，云雾缭绕，增添了几分神秘感。


In [ ]:
# 依赖：pip install openai pandas
import os
import pandas as pd
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor

# 建议用环境变量配置 API Key（PowerShell: $env:DASHSCOPE_API_KEY="你的key"）
client = OpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

SYSTEM_PROMPT = (
    "你是《山海经》研究专家和AI图像生成提示词专家。请基于形象,分类和描述生成适合AI绘画的简体中文prompt，"
    "如果有描述，详细扩写描述部分内容，不要有其他内容，如果没有则自由发挥，利用你的知识库搜索相关的神话学、考古学资料，"
    "参考古代插画风格和传统中国美学的彩色水墨风格,只返回prompt，以prompt开头。"
)

MAX_WORKERS = 6

def generate_prompt(name: str, category: str, description: str) -> str:
    """
    输入：名字、二级类目、描述
    输出：prompt 文本（以 'prompt' 开头）
    仅对请求发送做错误处理；失败时返回空字符串。
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"名字：{name}，分类：{category}，描述：{description or ''}"},
    ]
    try:
        resp = client.chat.completions.create(
            model="qwen-max",
            messages=messages,
            extra_body={"enable_search": True},
        )
        return (resp.choices[0].message.content or "").strip()
    except Exception as e:
        print(f"[ERROR] 请求失败：名字={name} 分类={category} 错误={e}")
        return ""

root_dir = "csv_by_category"

# 遍历所有 csv
for dirpath, dirnames, filenames in os.walk(root_dir):
    for fname in filenames:
        if not fname.lower().endswith(".csv"):
            continue

        csv_path = os.path.join(dirpath, fname)
        print(f"处理: {csv_path}")

        # 读取 CSV（统一用 utf-8-sig，便于兼容含中文的文件）
        df = pd.read_csv(csv_path, encoding="utf-8-sig")

        # 二级类目：优先取列「二级类目」，缺失则用当前文件名（去掉扩展名）
        if "二级类目" in df.columns:
            sec_cat = df["二级类目"].fillna("")
        else:
            inferred = os.path.splitext(fname)[0]
            sec_cat = pd.Series([inferred] * len(df))
            df["二级类目"] = sec_cat  # 顺便补齐一列，方便之后查看

        # 描述：没有就置空
        desc = df["描述"] if "描述" in df.columns else pd.Series([""] * len(df))

        # 写入新列并保存回原文件
        inputs = list(zip(df["名字"].astype(str), sec_cat.astype(str), desc.astype(str)))
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
            prompts = list(ex.map(lambda t: generate_prompt(*t), inputs))

        # 写入新列并保存回原文件
        df["prompt"] = prompts
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"  写入完成：{len(df)} 行")

处理: csv_by_category\乐舞\乐名.csv
  写入完成：4 行
处理: csv_by_category\乐舞\乐器.csv
  写入完成：5 行
处理: csv_by_category\乐舞\舞名.csv
  写入完成：5 行
处理: csv_by_category\人物\人名.csv
  写入完成：236 行
处理: csv_by_category\人物\尸.csv
  写入完成：11 行
处理: csv_by_category\人物\神名.csv
  写入完成：145 行
处理: csv_by_category\其他\其他.csv
  写入完成：12 行
处理: csv_by_category\其他\视肉.csv
  写入完成：1 行
处理: csv_by_category\动物\兽名.csv
  写入完成：207 行
处理: csv_by_category\动物\虫名.csv
  写入完成：43 行
处理: csv_by_category\动物\螺.csv
  写入完成：2 行
处理: csv_by_category\动物\贝.csv
  写入完成：5 行
处理: csv_by_category\动物\鱼名.csv
  写入完成：62 行
处理: csv_by_category\动物\鸟名.csv
  写入完成：140 行
处理: csv_by_category\动物\龟鳖.csv
  写入完成：10 行
处理: csv_by_category\器物\兵器.csv
  写入完成：11 行
处理: csv_by_category\器物\刑具.csv
  写入完成：1 行
处理: csv_by_category\器物\器物.csv
  写入完成：26 行
处理: csv_by_category\器物\服饰.csv
  写入完成：10 行
处理: csv_by_category\地名\山名.csv
  写入完成：666 行
处理: csv_by_category\地名\山系.csv
  写入完成：29 行
处理: csv_by_category\地名\建筑.csv
  写入完成：20 行
处理: csv_by_category\地名\森林.csv
  写入完成：9 行
处理: csv_by_category\地名\水名.csv
  写入完成：334

In [1]:
import os
import pandas as pd

root_dir = "csv_by_category"

# 遍历所有 csv
for dirpath, dirnames, filenames in os.walk(root_dir):
    for fname in filenames:
        if not fname.lower().endswith(".csv"):
            continue

        csv_path = os.path.join(dirpath, fname)
        print(f"处理: {csv_path}")

        # 读取 CSV
        df = pd.read_csv(csv_path, encoding="utf-8-sig")
        
        # 删除名字为空的行（包括 NaN 和空字符串）
        original_len = len(df)
        df = df[df["名字"].notna() & (df["名字"].astype(str).str.strip() != "")]
        
        # 保存回原文件
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        removed = original_len - len(df)
        print(f"  删除 {removed} 行，剩余 {len(df)} 行")

处理: csv_by_category\乐舞\乐名.csv
  删除 0 行，剩余 4 行
处理: csv_by_category\乐舞\乐器.csv
  删除 0 行，剩余 5 行
处理: csv_by_category\乐舞\舞名.csv
  删除 0 行，剩余 5 行
处理: csv_by_category\人物\人名.csv
  删除 34 行，剩余 201 行
处理: csv_by_category\人物\尸.csv
  删除 0 行，剩余 11 行
处理: csv_by_category\人物\神名.csv
  删除 23 行，剩余 122 行
处理: csv_by_category\其他\其他.csv
  删除 0 行，剩余 12 行
处理: csv_by_category\其他\视肉.csv
  删除 0 行，剩余 1 行
处理: csv_by_category\动物\兽名.csv
  删除 3 行，剩余 204 行
处理: csv_by_category\动物\虫名.csv
  删除 0 行，剩余 43 行
处理: csv_by_category\动物\螺.csv
  删除 0 行，剩余 2 行
处理: csv_by_category\动物\贝.csv
  删除 0 行，剩余 5 行
处理: csv_by_category\动物\鱼名.csv
  删除 0 行，剩余 62 行
处理: csv_by_category\动物\鸟名.csv
  删除 3 行，剩余 137 行
处理: csv_by_category\动物\龟鳖.csv
  删除 0 行，剩余 10 行
处理: csv_by_category\器物\兵器.csv
  删除 0 行，剩余 11 行
处理: csv_by_category\器物\刑具.csv
  删除 0 行，剩余 1 行
处理: csv_by_category\器物\器物.csv
  删除 0 行，剩余 26 行
处理: csv_by_category\器物\服饰.csv
  删除 0 行，剩余 10 行
处理: csv_by_category\地名\山名.csv
  删除 0 行，剩余 666 行
处理: csv_by_category\地名\山系.csv
  删除 0 行，剩余 29 行
处理: csv_by_categ

In [ ]:
#把csv汇总到一个总的csv里面